# Arbeiten mit Compute

Wenn Sie ein Skript als Azure Machine Learning-Auftrag ausführen, müssen Sie den Ausführungskontext für die Auftragsausführung festlegen. Eine Schlüsselkonfiguration ist das Computeziel, auf dem das Skript ausgeführt wird. Dies kann die lokale Arbeitsstation (in diesem Fall die Compute-Instanz) oder ein Remotecomputeziel wie der verwaltete Azure Machine Learning-Computecluster sein, der bedarfsgesteuert bereitgestellt wird.

In diesem Notebook erstellen Sie einen Computecluster und erkunden Computeziele für Aufträge.

## Vorbereitung

Sie benötigen die neueste Version des Pakets **azureml-ai-ml**, um den Code in diesem Notebook auszuführen. Führen Sie die folgende Zelle aus, um zu überprüfen, ob das Paket installiert ist.

> **Hinweis**:
> Wenn das Paket **azure-ai-ml** nicht installiert ist, führen Sie `pip install azure-ai-ml` aus, um es zu installieren.

In [ ]:
## Herstellen einer Verbindung mit Ihrem Arbeitsbereich

Sie können nun eine Verbindung mit Ihrem Arbeitsbereich herstellen, nachdem Sie die erforderlichen SDK-Pakete installiert haben.

Um eine Verbindung mit einem Arbeitsbereich herzustellen, benötigen Sie Bezeichnerparameter: eine Abonnement-ID, einen Ressourcengruppennamen und einen Arbeitsbereichsnamen. Der Ressourcengruppenname und Arbeitsbereichsname sind bereits für Sie ausgefüllt. Sie müssen lediglich die Abonnement-ID angeben, um den Befehl zu vervollständigen.

Um die erforderlichen Parameter zu finden, klicken Sie rechts oben in Studio auf das Abonnement und den Namen des Arbeitsbereichs. Rechts wird ein Bereich geöffnet.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Kopieren Sie die Abonnement-ID, und ersetzen Sie **YOUR-SUBSCRIPTION-ID** durch den Wert, den Sie kopiert haben. </p>

## Erstellen eines Computeclusters

In vielen Fällen reichen Ihre lokalen Computeressourcen möglicherweise nicht aus, um ein komplexes Experiment oder ein Experiment mit langer Ausführungszeit zu verarbeiten, das eine große Datenmenge verarbeiten muss. Sie sollten die Möglichkeit nutzen, Computeressourcen dynamisch in der Cloud zu erstellen und zu verwenden. Azure Machine Learning unterstützt eine Reihe von Computezielen, die Sie in Ihrem Arbeitsbereich definieren und für die Ausführung von Aufträgen verwenden können, wobei Sie für die Ressourcen nur dann zahlen, wenn Sie sie nutzen.

Sie können in [Azure Machine Learning Studio](https://ml.azure.com) mit dem Python SDK oder der Azure CLI einen Computecluster erstellen. Die folgende Codezelle überprüft Ihren Arbeitsbereich auf das Vorhandensein eines Computeclusters mit dem Namen `aml-cluster` und erstellt ihn, falls nicht vorhanden.

In [ ]:
Nachdem Sie einen Computecluster erstellt haben, können Sie die Konfiguration nur für Folgendes ändern:

- `min_instances`: Mindestanzahl von Knoten
- `max_instances`: Höchstanzahl von Knoten
- `idle_time_before_scale_down`: Leerlaufzeit vor dem Herunterskalieren

Derzeit kann Ihr Computecluster `aml-cluster` nur auf maximal einen Knoten skaliert werden. Lassen Sie uns das in 2 ändern, um parallele Compute-Instanzen zu ermöglichen.

In [ ]:
Wenn der Computecluster aktualisiert wird, können Sie seine Konfiguration überprüfen, indem Sie seine Attribute ausgeben.

## Erstellen eines Skripts zum Trainieren eines Modells

Um ein Modell zu trainieren, erstellen Sie zuerst im Ordner **src** das Skript **diabetes_training.py**. Das Skript nutzt die Datei **diabetes.csv** im Ordner, in dem sich die Trainingsdaten befinden.

In [ ]:
## Ausführen eines Auftrags in einem Computecluster

Jetzt können Sie den Auftrag im von Ihnen erstellten Computecluster ausführen.

> **Hinweis**:
> Der Start des Auftrags nimmt einige Zeit in Anspruch, da der Computecluster von 0 auf 1 Knoten skaliert werden muss. Sobald der Computecluster bereit ist, wird das Skript ausgeführt. Wenn der Auftrag abgeschlossen ist, wird der Computecluster wieder auf 0 Knoten herunterskaliert. Sie können den Status des Computeclusters auf der Seite **Compute** überprüfen.

After you've created a compute cluster, you can only change the configuration for:

- `min_instances`: Minimum number of nodes
- `max_instances`: Maximum number of nodes
- `idle_time_before_scale_down`: Idle time before scale down

Currently, your compute cluster `aml-cluster` can only scale do a maximum of one node. Let's change that to two, to allow for parallel compute.

In [ ]:
from azure.ai.ml.entities import AmlCompute

cluster_scale = AmlCompute(
    name="cpu-cluster",
    max_instances=2,
)
ml_client.begin_create_or_update(cluster_scale)

When the compute cluster is updated, you can verify its configuration by printing its attributes.

In [ ]:
cpu_cluster = ml_client.compute.get("cpu-cluster")

print (
        f"AMLCompute with name {cpu_cluster.name} has a maximum of {cpu_cluster.max_instances} nodes"
    )

## Create a script to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

## Run a job on a compute cluster

Now, you're ready to run the job on the compute cluster you created.

> **Note**:
> The job will take some time to start as the compute cluster will need to scale from zero to one node. Once the compute cluster is ready, the script will be run. When the job has finished, the compute cluster will scale back down to zero nodes. You can review the compute cluster's status in the **Compute** page.

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="cpu-cluster",
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)